# Importing the Data

**Goal**: Classify attacks assuming a multiclass classification system has alerted an attack.

In [1]:



import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


from sklearn.metrics import classification_report, accuracy_score

In [2]:



  #change dir to your project folder

In [3]:
#cleaned data without non-attack values
X = pd.read_csv('~/datasets/Iris.csv', header=None, usecols=[i for i in range(4)])
Y = pd.read_csv('~/datasets/Iris.csv', header=None, usecols=[4])

# Feature Engineering

In [4]:
# cluster and score
from sklearn.cluster import Birch
from sklearn.metrics.cluster import fowlkes_mallows_score as scorer
score = []
for i in range(len(X.columns)): # loop number of features
  K = Birch(n_clusters=len(Y[Y.columns[0]].unique()))
  pred = K.fit_predict(X.iloc[:, [i]].values)
  s = scorer(Y[Y.columns[0]].values,pred)
  score.append(s)
  

# Rank the features and sort

s2 = score
np.asarray(s2)

s1 = []
for i in range(len(X.columns)):
  s1.append(i)
  
np.asarray(s1)

li = list(zip(s1, s2))

def sortSecond(val): 
    return val[1] 
  
li.sort(key = sortSecond, reverse=True) 



    
# Create a copy of X dataframe with columns sorted by score

titles = []

for i in range(len(X.columns)):
  p = X.columns[li[i][0]]
  titles.append(p)


X1 = pd.DataFrame(columns=titles)

for i in range(len(X.columns)):
  X1[X1.columns[i]] = X[X.columns[li[i][0]]]
  
  

# imports
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier as rf
from sklearn.metrics import accuracy_score

# Recursive Feature Elemination from # of features to 0 and keep the accuracy score of each

accuracy = []
X2 = X1.copy()

# for i in range(len(X1.columns)-1,-1,-1):
for i in range(len(X1.columns)-1): 
  x_train, x_test, y_train, y_test = train_test_split(X1, Y, test_size=0.2, random_state=11)
  X1.drop(X1.columns[len(X1.columns)-1], axis=1, inplace=True)
  clf = rf(random_state=0, n_jobs=-1)
  clf.fit(x_train, y_train)
  y_pred = clf.predict(x_test)
  accuracy.append(accuracy_score(y_test, y_pred)) 
  

  
# best score calcuation
index = accuracy.index(max(accuracy))

X = X2.iloc[:,0:len(X.columns)-index]

/Users/jimenez/anaconda3/lib/python3.7/site-packages/sklearn/cluster/birch.py:627: ConvergenceWarning: Number of subclusters found (1) by Birch is less than (3). Decrease the threshold.
  % (len(centroids), self.n_clusters), ConvergenceWarning)
/Users/jimenez/anaconda3/lib/python3.7/site-packages/sklearn/cluster/birch.py:627: ConvergenceWarning: Number of subclusters found (2) by Birch is less than (3). Decrease the threshold.
  % (len(centroids), self.n_clusters), ConvergenceWarning)
/Users/jimenez/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/jimenez/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:64: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/Users/jimenez/anaconda3/lib/python

# Performance Analysis

In [5]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=11)

In [6]:
from sklearn.ensemble import RandomForestClassifier as rf
clf = rf(random_state=0, n_jobs=-1)
clf.fit(x_train, y_train)

/Users/jimenez/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/jimenez/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
                       oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [7]:

x = list(zip(X.columns, clf.feature_importances_))

def sortSecond(val): 
    return val[1] 
  
x.sort(key = sortSecond, reverse=True) 

for i in range(len(X.columns)):
    print(x[i])
    if (i==len(X.columns)-1):
      print("features",i+1)

(3, 0.48277755005284784)
(2, 0.47361829316801873)
(0, 0.03611610054457399)
(1, 0.007488056234559509)
features 4


In [8]:
print(clf.score(x_train, y_train), "train")

0.9916666666666667 train


In [9]:
y_pred = clf.predict(x_test)

print("Classification\n")
print(classification_report(y_test,y_pred, digits=4))

Classification

                 precision    recall  f1-score   support

    Iris-setosa     1.0000    1.0000    1.0000         9
Iris-versicolor     0.8333    1.0000    0.9091        10
 Iris-virginica     1.0000    0.8182    0.9000        11

       accuracy                         0.9333        30
      macro avg     0.9444    0.9394    0.9364        30
   weighted avg     0.9444    0.9333    0.9330        30

